### 1.   download all necessairy dependencies.



In [ ]:
%%capture
!pip install livelossplot

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
import random
from livelossplot import PlotLosses
from keras.layers import SimpleRNN

### 2.   download the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
# opening the file
train_file = open('/content/drive/MyDrive/music-generator/train.json')

json_data = json.load(train_file)
train_list = []
for item in json_data:
  train_list.append(item["abc notation"])

train_text = "\n\n".join(train_list)
print(train_text[:500])

# closing the file
train_file.close()

> Test our data

In [ ]:
from music21 import converter
song = converter.parse(train_list[random.randint(0,100)])
song.show('midi')

print(train_list[random.randint(0,100)])
# song.write('midi', fp='output.mid')

## 3. building the tokenizer

In [ ]:
class Tokenizer:
    def __init__(self, text_data, eost=None):
        self.tokens = sorted(set(train_text))
        self.stoi = {s:i for i, s in enumerate(self.tokens)}
        self.vocab_size = len(self.tokens)
        if eost:
            self.stoi["%"] = self.vocab_size
            self.vocab_size +=1
        self.itos = {i:s for s, i in self.stoi.items()}

    def encode(self, songs_string):
      enc = []
      for s in list(songs_string):
          enc.append(self.stoi[s])
      return enc

    def decode(self, songs_arr):
      dec = []
      for i in songs_arr:
          dec.append(self.itos[i])
      return "".join(dec)

tokenizer = Tokenizer(train_text)
print(f"size of the vocabulary : {tokenizer.vocab_size}")
# print(tokenizer.itos.keys())

In [ ]:
enc = tokenizer.encode("E2 EF E2 B2 |1 efe^d e2 e2")
print(enc)
print(tokenizer.decode(enc))

## 4. tokenize abc annotations

In [ ]:
train_tokens = tokenizer.encode(train_text)
train_tokens[:10]

## 5. let's now create batchs

In [ ]:
def get_batch(train_tokens, block_size, batch_size):
  idx = np.random.choice(len(train_tokens)-block_size-1, batch_size)

  x = np.reshape([train_tokens[i:i+block_size] for i in idx], [batch_size, block_size])
  y = np.reshape([train_tokens[i+1:i+block_size+1] for i in idx], [batch_size, block_size])
  return x, y

get_batch(train_tokens, block_size=8, batch_size=3)

## 5. Building the model

> let's define LSTM layer

In [ ]:
def RNNs_Builder(rnn_units):
    return SimpleRNN(
        rnn_units,
        return_sequences=True,
        activation='tanh',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        recurrent_initializer='orthogonal',
        bias_initializer='zeros',
        stateful=True,
    )

rnn = RNNs_Builder(32)
print(f"RNN Units: {rnn.units}")

> let's stack lstms together then add fcn

In [ ]:
def Model_Builder(vocab_size, emb_size, lstm_units, batch_size):
    model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,
                                emb_size,
                                batch_input_shape=[batch_size, None]),
      RNNs_Builder(lstm_units),
      tf.keras.layers.Dense(units=vocab_size,
                            activation='relu',
                            kernel_initializer='glorot_uniform',
                            bias_initializer='zeros')
    ])

    return model

In [ ]:
model = Model_Builder(tokenizer.vocab_size, emb_size=64, lstm_units=128, batch_size=16)
model.summary()

> check input, output dims

In [ ]:
input, _ = get_batch(train_tokens, block_size=8, batch_size=16)
output = model(input)
print(f"Input shape: [batch_size, sequence_length] <==> {input.shape}")
print(f"output shape: [batch_size, sequence_length, vocab_size] <==> {output.shape}")

## 6. Training the model

> Hyperparameters

In [ ]:
chpt_dir = './content/model_checkpoints'
chpt_prefix = os.path.join(chpt_dir, "lstm_chpts")

num_steps = 2000
batch_size = 64
block_size = 200
vocab_size = tokenizer.vocab_size
lr = 5e-3
emb_size = 256
lstm_units = 1024
beta1, beta2=0.9, 0.99
eps=1e-06

> loss functions (We have defined it separately because we want to experiment with the loss function as well.)

In [ ]:
def loss_function(y, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(y, logits, from_logits=True)

> initialization

In [ ]:
model = Model_Builder(vocab_size=vocab_size,
                        emb_size=emb_size,
                        lstm_units=lstm_units,
                        batch_size=batch_size)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr,
    beta_1=beta1,
    beta_2=beta2,
    epsilon=eps,
    name='Adam',
)

# optimizer = tf.keras.optimizers.experimental.Adagrad(
#     learning_rate=lr,
#     initial_accumulator_value=0.1,
#     epsilon=1e-07,
#     ema_momentum=0.99,
#     ema_overwrite_frequency=None,
#     jit_compile=True,
#     name='Adagrad',
#     )

> one step of training

In [ ]:
@tf.function
def train_iter(xb, yb):
  with tf.GradientTape() as tape:
    logits = model(xb)
    loss = loss_function(yb, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

In [ ]:
liveloss = PlotLosses()
logs = {}

# clear previous tqdm instances
if hasattr(tqdm, '_instances'): tqdm._instances.clear()

# start training
for iter in tqdm(range(num_steps)):
  xb, yb = get_batch(train_tokens, block_size=block_size, batch_size=batch_size)

  loss = train_iter(xb, yb)

  logs["train"] = loss.numpy().mean()
  liveloss.update(logs)
  liveloss.draw()

  if iter % 10 == 0:
    model.save_weights(chpt_prefix)

## 7. let's load our model and re-build it (with batch=1)

In [ ]:
infer_model = Model_Builder(vocab_size, emb_size, lstm_units, batch_size=1)

# Restore the model weights for the last checkpoint after training
infer_model.load_weights(tf.train.latest_checkpoint(chpt_dir))
infer_model.build(tf.TensorShape([1, None]))

infer_model.summary()

## 8. Convert ABC annotations to our music

In [ ]:
def generate_ABC(model, start, length=1000):
  input_eval = tf.expand_dims(tokenizer.encode(start), 0)

  generated = []

  model.reset_states()
  tqdm._instances.clear()

  for i in tqdm(range(length)):
      preds = model(input_eval)
      preds = tf.squeeze(preds, 0)
      id = tf.random.categorical(preds, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([id], 0)
      generated.append(tokenizer.decode([id]))
  return (start + ''.join(generated))

In [ ]:
ABC = generate_ABC(infer_model, start="M:4/4\nK:A\nA", length=500)
print(ABC)

## 9. Convert ABC annotation to audio

In [ ]:
from music21 import converter
song = converter.parse(ABC)
song.show('midi')

# song.write('midi', fp='output.mid')

***Congratulations! You can now relish your very own melodies.:***

<img width="100%" src="https://github-production-user-asset-6210df.s3.amazonaws.com/89405673/247540874-82a528db-da22-4d14-a3c6-5360655384da.gif"/>